<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Langchain_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agents 🤖
Agents are like "tools" for LLMs. They allow a LLM to access Google search, perform complex calculations with Python, and even make SQL queries.

In this notebook we'll explore agents and how to use them in LangChain.

We'll start by installing the prerequisite libraries that we'll be using in this example.

In [1]:
!pip install -qU langchain openai google-search-results wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.5/626.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.0 MB/s eta 0:00:00


In [2]:
from getpass import getpass
OPENAI_API_KEY = getpass()

··········


In [3]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

## Counting tokens in each call

In [4]:
from langchain.callbacks import get_openai_callback
def countTokens(agent,query):
  with get_openai_callback() as cb:
    result =agent(query)
  print(f"Spent a total of {cb.total_tokens} tokens")
  return result

## What is an agent?
####Definition: 
The key behind agents is giving LLM's the possibility of using tools in their workflow. This is where langchain departs from the popular chatgpt implementation and we can start to get a glimpse of what it offers us as builders. Until now, we covered several building blocks in isolation. Let's see them come to life.

The official definition of agents is the following:

Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

In this edition we will cover what we may call 'generic' agents which really able to perform many meta tasks. There are other more specific agents that are tuned for different tasks (called 'agent-toolkits'), but we will cover those in a future edition.

To use agents we need three main components:

* Large Language Models (LLMs)
* Tools
* The Agents themselves

In [5]:
from langchain.chains import LLMChain,LLMMathChain
from langchain.agents import Tool
#
llm_math = LLMMathChain(llm=llm)
#
math_tool = Tool(name="calculator",
                 func= llm_math.run ,
                 description = "Useful for when we need to answer questions about math.")
#
tools =[math_tool]

In [6]:
tools[0].name, tools[0].description

('calculator', 'Useful for when we need to answer questions about math.')

##Initialize agent

In [7]:
from langchain.agents import initialize_agent
zeroshot_agent = initialize_agent(agent ='zero-shot-react-description',
                                  tools=tools,
                                  llm=llm,
                                  verbose=True ,
                                  max_iterations=3)

In [8]:
zeroshot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [9]:
zeroshot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
 We need to calculate the total number of apples.
Action: calculator
Action Input: 4 + (2.5 * 8)
Observation: Answer: 24.0
Thought: We now have the total number of apples.
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

If we'd like the agent to answer this "general knowledge" question. We can, we just need to link it to an LLM tool.

We initialize the tool like so:

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
prompt = PromptTemplate(input_variables=["query"],
                        template="{query}")
llm_chain = LLMChain(llm=llm,
                     prompt=prompt)


## initialize language tool

In [11]:
llm_tool = Tool(name="language_tool",
                func = llm_chain.run,
                description = "Use this tool for general purpose queries")
tools.append(llm_tool)

##Re-initialize the agent with 2 tools

In [12]:
zeroshot_agent = initialize_agent(agent ="zero-shot-react-description",
                                   tools=tools,
                                   llm=llm,
                                   verbose=True,
                                   max_iterations=6)

In [13]:
zeroshot_agent("What is the capital of India?")



> Entering new AgentExecutor chain...
 I need to find out what the capital of India is.
Action: language_tool
Action Input: "What is the capital of India?"
Observation: 

The capital of India is New Delhi.
Thought: I now know the final answer.
Final Answer: The capital of India is New Delhi.

> Finished chain.


{'input': 'What is the capital of India?',
 'output': 'The capital of India is New Delhi.'}

In [14]:
for query in ["What is the capital of India?","what is (4.5*2.1)^2.2?"]:
  zeroshot_agent(query)



> Entering new AgentExecutor chain...
 I need to find out what the capital of India is.
Action: language_tool
Action Input: Capital of India
Observation: 

Delhi is the capital of India.
Thought: I now know the final answer.
Final Answer: Delhi is the capital of India.

> Finished chain.


> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: calculator
Action Input: 4.5*2.1^2.2
Observation: Answer: 23.01945102298866
Thought: I now know the final answer
Final Answer: 23.01945102298866

> Finished chain.


## Create database
We will use the agents to interact with a small sample database of stocks. 

* This is just a dummy tool we will build for illustrative purposes. Let's create it.

In [15]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [16]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),    
)

In [42]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", pool_size=10,pool_pre_ping=True)
metadata_obj.create_all(engine)

In [18]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [34]:
observations = [
    [11, 'ABC1', 200, datetime(2023, 1, 1)],
    [12, 'ABC1', 208, datetime(2023, 1, 2)],
    [13, 'ABC1', 232, datetime(2023, 1, 3)],
    [14, 'ABC1', 225, datetime(2023, 1, 4)],
    [15, 'ABC1', 226, datetime(2023, 1, 5)],
    [16, 'XYZ1', 810, datetime(2023, 1, 1)],
    [17, 'XYZ1', 803, datetime(2023, 1, 2)],
    [18, 'XYZ1', 798, datetime(2023, 1, 3)],
    [19, 'XYZ1', 795, datetime(2023, 1, 4)],
    [20, 'XYZ1', 791, datetime(2023, 1, 5)],
]

In [35]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0], 
    stock_ticker=obs[1], 
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

## Insert rows into the table

In [36]:
for obs in observations:
    insert_obs(obs)

In [37]:
from langchain.sql_database import SQLDatabase
from langchain.chains import SQLDatabaseChain
#

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

## Create a db Tool

In [38]:
db_tool = Tool(name="stock_db",
               func = sql_chain.run,
               description="Useful for when you need to answer questions about stocks and their prices.")

In [39]:
tools.append(db_tool)

In [40]:
tools[2].name

'stock_db'

## Initialize the agent again

In [25]:
#Important Note: When interacting with agents it is really important to set the max_iterations parameters because agents can get stuck in infinite loops that consume plenty of tokens. The default value is 15 to allow for many tools and complex reasoning but for most applications you should keep it much lower.

from langchain.agents import initialize_agent

zeroshot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=6,
)

In [91]:
result = countTokens(
    zeroshot_agent, 
    "Use sqlite syntax to answer this query:What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th?"
)



> Entering new AgentExecutor chain...
 I need to use the stock_db tool to answer this question.
Action: stock_db
Action Input: SELECT (price_ABC/price_XYZ) AS ratio_Jan3, (price_ABC/price_XYZ) AS ratio_Jan4 FROM stocks WHERE date = 'January 3rd' OR date = 'January 4th'

> Entering new SQLDatabaseChain chain...
SELECT (price_ABC/price_XYZ) AS ratio_Jan3, (price_ABC/price_XYZ) AS ratio_Jan4 FROM stocks WHERE date = 'January 3rd' OR date = 'January 4th'
SQLQuery: SELECT (price/ (SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND date = '2023-01-03')) AS ratio_Jan3, (price/ (SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND date = '2023-01-04')) AS ratio_Jan4 FROM stocks WHERE stock_ticker = 'ABC' AND (date = '2023-01-03' OR date = '2023-01-04')
SQLResult: [(0.2907268170426065, 0.2918238993710692), (0.2819548872180451, 0.2830188679245283)]
Answer: The ratio of ABC to XYZ on January 3rd was 0.2907 and on January 4th was 0.2918.
> Finished chain.

Observation:  The ratio of ABC 

In [92]:
for query in ["What is the capital of India?",
              "what is (4.5*2.1)^2.2?",
              "What is the price for stock_ticker XYZ on Januray 5th ?",
              "What is the maximum value of stock prices for stock_ticker ABC ?",
              "Use sqlite syntax to answer this query:What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th?"
              ]:
  zeroshot_agent(query)



> Entering new AgentExecutor chain...
 India is a country, so I should use a language tool to find the answer.
Action: language_tool
Action Input: capital of India
Observation: 

Delhi is the capital of India.
Thought: I now know the final answer.
Final Answer: Delhi is the capital of India.

> Finished chain.


> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: calculator
Action Input: 4.5*2.1^2.2
Observation: Answer: 23.01945102298866
Thought: I now know the final answer
Final Answer: 23.01945102298866

> Finished chain.


> Entering new AgentExecutor chain...
 I need to find the price of a stock on a certain date
Action: stock_db
Action Input: stock_ticker XYZ, date January 5th

> Entering new SQLDatabaseChain chain...
stock_ticker XYZ, date January 5th
SQLQuery: SELECT price FROM stocks WHERE stock_ticker = "XYZ" AND date = "2023-01-05"
SQLResult: [(791.0,)]
Answer: The price of stock_ticker XYZ on January 5th is 791.0.
> Finished chain.

Observati

In [93]:
print(zeroshot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

calculator: Useful for when we need to answer questions about math.
language_tool: Use this tool for general purpose queries
stock_db: Useful for when you need to answer questions about stocks and their prices.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [calculator, language_tool, stock_db]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


## How are agents different than chains?

If we look at the agent's logic and the prompt we have just printed we will see some clear differences. 

* First, we have the tools which are included in the prompt. 
* Second we have a thought process which before was immediate in chains but now involves a 
* 'thought', 
* 'action', 
* 'action input', 
* 'observation' sequence. 

What is this all about?

Suffice it to say for now that the LLM now has the ability to 'reason' on how to best use tools to solve our query and can combine them in intelligent ways with just a brief description of each of them. 

#### What is  'agent_scratchpad'?

Well, that is where we will be appending every thought or action that the agent has already performed. 

In this way, at each point in time, the agent will know what it has found out and will be able to continue its thought process. 

In other words, after using a tool it adds its thoughts and observations to the scratchpad and picks up from there.

## Agent type #2: Conversational React


* The zero shot agent is really interesting but, as we said before, it has no memory. 

* What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? 

* For that we have the conversational react agent.

We will use the same tools as we specified earlier:

In [84]:
from langchain.agents import load_tools
tools = load_tools(
    ["llm-math"], 
    llm=llm
)

In [86]:
tools.append(db_tool)

In [98]:
tools.append(llm_tool)

In [102]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [103]:
conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [97]:
result = countTokens(
    conversational_agent, 
    "Use sqlite syntax to answer this query:Please provide me the stock prices for ABC on January the 1st"
                     )



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: stock_db
Action Input: stock prices for ABC on January the 1st

> Entering new SQLDatabaseChain chain...
stock prices for ABC on January the 1st
SQLQuery: SELECT price FROM stocks WHERE stock_ticker = "ABC1" AND date = "2023-01-01"
SQLResult: [(200.0,)]
Answer: The stock price for ABC on January the 1st was 200.0.
> Finished chain.

Observation:  The stock price for ABC on January the 1st was 200.0.
Thought: Do I need to use a tool? No
AI: Is there anything else I can help you with?

> Finished chain.
Spent a total of 1850 tokens


In [99]:
result = countTokens(
    conversational_agent, 
    "what is the diameter of a circle having a radius as 30 cm"
                     )



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: The diameter of a circle with a radius of 30 cm is 60 cm.

> Finished chain.
Spent a total of 511 tokens


In [104]:
conversational_agent("what is the capital of norway ?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: The capital of Norway is Oslo.

> Finished chain.


{'input': 'what is the capital of norway ?',
 'chat_history': '',
 'output': 'The capital of Norway is Oslo.'}

* As we can see below, the prompt is similar but it includes a great prelude of instructions that make it an effective assistant as well + a spot for including the chat history from the memory component:

In [105]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

## Agent type #3: React Docstore
This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. 

It will have two and only two tools at its disposal: **'Search' and 'Lookup'**.

* With 'Search' it will bring up a relevant article and 
* with 'Lookup' the agent will find the right piece of information in the article. 

This is probably easiest to see in an example:

In [106]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools_doc = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [111]:
docstore_agent = initialize_agent(
    tools_doc, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=6
)

In [109]:
docstore_agent("What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (; c. 287 – c. 212 BC) was a Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.Archimedes' other mathemat

{'input': "What were Archimedes' last words?",
 'output': '"Do not disturb my circles"'}

In short, it contains several examples of the 
**Question > Thought > Action > Observation loop**, that include the Search and Lookup tools.

In [112]:
countTokens(docstore_agent,"What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (; c. 287 – c. 212 BC) was a Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.Archimedes' other mathemat

{'input': "What were Archimedes' last words?",
 'output': 'Agent stopped due to iteration limit or time limit.'}

## Agent type #4: Self Ask with Search
This is the first-choice agent to use when using LLM's to extract information with a search engine. 

The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

In [114]:
from langchain import OpenAI,SerpAPIWrapper
from langchain.agents import initialize_agent,Tool
#
serp_api_key = getpass()
search = SerpAPIWrapper(serpapi_api_key=serp_api_key)

··········


In [115]:
tools_serp = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

In [116]:
self_ask_with_search = initialize_agent(tools_serp, 
                                        llm, 
                                        agent="self-ask-with-search", 
                                        verbose=True)

In [118]:
self_ask_with_search("When did Indian gain its freedom ?")



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who was the leader of India during its independence?
Intermediate answer: Mohandas GANDHI
Follow up: When did India gain its independence?
Intermediate answer: August 15, 1947
So the final answer is: August 15, 1947

> Finished chain.


{'input': 'When did Indian gain its freedom ?', 'output': 'August 15, 1947'}

In [117]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

* https://python.langchain.com/en/latest/modules/agents/how_to_guides.html